In [ ]:
import pandas as pd
import re, math
import seaborn as sns

from matplotlib import pyplot as plt
from glob import glob
from pprint import pprint

pd.options.display.float_format = "{:,.2f}".format

# Config Session

In [2]:
PATH_OUTPUT = 'data/dataset_carnes_kg.csv'

PROTEINS = {
    1: 'CARNES',
    2: 'CARNES ULTRAPROCESSADAS',
    3: 'PESCADOS',
    4: 'OVOS',
    5: 'SUBSTITUTOS DE CARNE'
}

CODES = {
    11: 'CARNE BOVINA',
    12: 'CARNE SUINA',
    13: 'OUTRAS CARNES',
    14: 'AVES',
    15: 'CARNES SECAS E SALGADAS',
    16: 'VÍSCERAS',
    17: 'PREPARAÇÕES QUE INCLUEM CARNES BOVINAS',
    17: 'PREPARAÇÕES QUE INCLUEM OUTRAS CARNES',
    21: 'ALMÔNDEGAS, HAMBURGERS E NUGGETS',
    22: 'EMBUTIDOS, FRIOS E CONSERVAS',
    23: 'PREPARAÇÕES QUE INCLUEM CARNES UP',	
    31: 'PESCADOS FRESCOS',
    32: 'PESCADOS EM CONSERVA',
    33: 'PESCADOS SALGADOS',
    34: 'PREPARAÇÕES QUE INCLUEM PESCADOS',
    41: 'OVOS INTEIROS',
    42: 'REFEIÇÕES QUE INCLUEM OVOS',
    51: 'PTS - CARNE DE SOJA',
    52: 'PREPARAÇÕES QUE INCLUEM CARNE DE SOJA'
}

CORTES = {
    111: 'CORTE TRASEIRO',
    112: 'CORTE DIANTEIRO'
}

PREMISSAS = {
    'ovos': {
        'peso_un_ovo': float(0.05), # peso de um ovo médio 0,05 kg
        'peso_un_ovo_grande': float(0.06), # peso de um ovo grande 0,06 kg
        'BANDEJA': 30, # uma bandeja tem x unidades
        'PLACA': 30,
        'DZ': 12,
        'CARTELA': 15,
        'PEN': 30,
        'CX': [30, 12] # 1 caixa tem 30 dúzias de ovos
    },
    'sardinha':{
        'LATA': 0.125
    }
}


# Input Data

In [3]:
df_dataset_beef = pd.read_parquet('data/dataset_carnes_v2.parquet')

# Functions

In [9]:
def standardize_unit(unidade):
    """
    Padroniza as unidades de medida com base em padrões gerais do mercado e abreviações.
    
    Parâmetro:
        unidade (str): O valor da unidade a ser padronizado.
    
    Retorna:
        str: Unidade padronizada.
    """
    unidade = str(unidade).strip().upper()
    
    # Remover asteriscos, vírgulas, e espaços extras
    unidade = re.sub(r'[\*\s,]+', '', unidade)
    

    padroes = {
        r'^(KG|KGS?|KGI|KILOGRAMA|KILOS?|KGM|KF|KU)$': 'KG',
        r'^(GRAMA|GRAMAS|G|GM)$': 'GRAMA',
        r'^(TON|TONELADA|TNS?|TONS?)$': 'TON',
        r'^(LITRO|LTR|LTS?|LITROS?)$': 'LITRO',
        r'^(ML|MILILITRO|MILILITROS)$': 'ML',
        r'^(CX|CAIXA|CAIXAS|CXS?|CXA?)$': 'CX',
        r'^(UN|UND|UNIT|UNID|UNIDADES?|UNS?|UID)$': 'UN',
        r'^(DZ|DÚZIA|DÚZIAS|DOZ)$': 'DZ',
        r'^(BDJ|BANDEJ|BANDEJA|BANDEJAS|BAN|BAND|BJ?)$': 'BANDEJA',
        r'^(PACOTE|PCT|PKG|PCTS?|PACOTES)$': 'PACOTE',
        r'^(CART|CAR|CARTEL|CARTELA|CARTELAS|CA|CAR|CARTS|CT|CARTONADO|CRT|CARTON?)$': 'CARTELA',
        r'^(FD|FARDO|FARDOS)$': 'FARDO',
        r'^(PL|pl|placa?)$': 'PLACA',
        r'^(ALIM|ALIMENTAÇÃO|ALIMENTAÇÕES)$': 'ALIMENTACAO',
        r'^(OUTROS|OUT|OTHER)$': 'OUTROS',
        r'^(LATA|LATAS)$': 'LATA'
    }
    

    for padrao, padronizado in padroes.items():
        if re.match(padrao, unidade):
            return padronizado

    return unidade

def convert_based_on_description(item, qtd, qt_kg):
    item = item.upper()
    match = re.search(r'(\d+)?\s*(X|C/)?\s*\.?(\d+\s*\d*)(KG|G|GR)', item)  
    peso_total = 0
    
    if match:
        quantidade = int(match.group(1)) if match.group(1) else 1  # Captura quantidade (opcional)
        peso_unitario = int(match.group(3).replace(' ', ''))  # Remove espaços entre os números
        unidade = match.group(4)
        print(item, '=>', quantidade, peso_unitario, unidade)        
        # Conversão para KG
        peso_total = int(quantidade) * (float(peso_unitario) / 1000) if 'G' in unidade else float(peso_unitario)
    if peso_total != 0:
        qt_kg = peso_total * qtd
    return qt_kg

def convert_eggs_kg(item, unidade, qtd):
    ovo_grande = PREMISSAS['ovos']['peso_un_ovo_grande']
    ovo_normal = PREMISSAS['ovos']['peso_un_ovo']

    qt_ovo_bjd = PREMISSAS['ovos']['BANDEJA']
    qt_ovo_plc = PREMISSAS['ovos']['PLACA']
    qt_ovo_dz = PREMISSAS['ovos']['DZ']
    qt_ovo_ctl = PREMISSAS['ovos']['CARTELA']
    qt_ovo_pen = PREMISSAS['ovos']['PEN']
    qt_ovo_cx = PREMISSAS['ovos']['CX'][0] * PREMISSAS['ovos']['CX'][1]

    match unidade:
        case 'UN':
            return qtd * ovo_grande if 'GRANDE' in item else qtd * ovo_normal
        case 'BANDEJA':
            return (qtd * qt_ovo_bjd) * ovo_grande if 'GRANDE' in item else (qtd * qt_ovo_bjd) * ovo_normal
        case 'PLACA':
            return (qtd * qt_ovo_plc) * ovo_grande if 'GRANDE' in item else (qtd * qt_ovo_plc) * ovo_normal
        case 'DZ':
            return (qtd * qt_ovo_dz) * ovo_grande if 'GRANDE' in item else (qtd * qt_ovo_dz) * ovo_normal
        case 'CARTELA':
            return (qtd * qt_ovo_ctl) * ovo_grande if 'GRANDE' in item else (qtd * qt_ovo_ctl) * ovo_normal
        case 'PEN':
            return (qtd * qt_ovo_pen) * ovo_grande if 'GRANDE' in item else (qtd * qt_ovo_pen) * ovo_normal
        case 'CX':
            return (qtd * qt_ovo_cx) * ovo_grande if 'GRANDE' in item else (qtd * qt_ovo_cx) * ovo_normal      
        case _:
            return 0
    
def convert_others_kg(row):
    
    unit = row['un_adj'].upper()
    item = row['item'].upper()
    qt = row['qt']
    vu = float(row['valor_un'])

    # Rule for sardinha in LATA
    if 'SARDINHA' in item and 'LATA' in unit:
        return qt * PREMISSAS['sardinha']['LATA']  # Sardinha lata = 125g    
    elif ('GRAMA' in unit and 'PROTEINA TEXTURIZADA DE SOJA' in item) and \
        (
            (qt >= 300 and qt <= 400 and vu >= 6 and vu <= 15) or 
            (qt >= 100 and qt <= 300 and vu >= 2.5 and vu <= 5)
        ):
        return qt * 0.001  # convert it to kg
    else:
        return qt

# Standardize values

In [10]:
# arruma e padroniza as unidades já existentes
df_dataset_beef['un_adj'] = df_dataset_beef['un'].apply(standardize_unit)

# ajustar unidade UN
df_dataset_beef.loc[
    (df_dataset_beef['un_adj'] == 'FARDO') &
    (df_dataset_beef['item'].str.contains('unidade', case=False)),
    'un_adj'
] = 'UN'

# ajustar unidade UN
df_dataset_beef.loc[
    (df_dataset_beef['un_adj'] == 'FARDO') &
    (df_dataset_beef['item'].str.contains('OVO DE GALINHA-FORMA', case=False)),
    'un_adj'
] = 'BANDEJA'


In [ ]:
df_dataset_beef['qt_kg'] = 0

df_dataset_beef['qt_kg'] = df_dataset_beef.apply(
	lambda serie: serie['qt'] if serie['un_adj'] == 'KG' else serie['qt_kg'], 
	1
)

df_dataset_beef['qt_kg'] = df_dataset_beef.apply(lambda serie: convert_based_on_description(serie['item'], serie['qt'], serie['qt_kg']), 1)
df_dataset_beef['qt_kg'] = df_dataset_beef.apply(lambda serie: convert_others_kg(serie), 1)

df_dataset_beef['qt_kg'] = df_dataset_beef.apply(
	lambda serie: convert_eggs_kg(serie['item'] ,serie['un_adj'], serie['qt'])
    	if serie['codigo'] in [41.0, 42.0] else 0, 
	1
)

In [ ]:

df_dataset_beef.to_csv(PATH_OUTPUT)
